In [1]:
import spacy

In [2]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import jsonpickle
import json
import random
import tweepy
import spacy
import time
from datetime import datetime

In [3]:
class Tweets:
    
    
    def __init__(self,term="",corpus_size=100):
        self.tweets={}
        if term !="":
            self.searchTwitter(term,corpus_size)
                
    def searchTwitter(self,term,corpus_size):
        searchTime=datetime.now()
        while (self.countTweets() < corpus_size):
            new_tweets = api.search(term,lang="en",count=10)
            for nt_json in new_tweets:
                nt = nt_json._json
                if self.getTweet(nt['id_str']) is None and self.countTweets() < corpus_size:
                    self.addTweet(nt,searchTime,term)
            time.sleep(5)
                
    def addTweet(self,tweet,searchTime,term="",count=0):
        id = tweet['id_str']
        if id not in self.tweets.keys():
            self.tweets[id]={}
            self.tweets[id]['tweet']=tweet
            self.tweets[id]['count']=0
            self.tweets[id]['searchTime']=searchTime
            self.tweets[id]['searchTerm']=term
        self.tweets[id]['count'] = self.tweets[id]['count'] +1
        
    def getTweet(self,id):
        if id in self.tweets:
            return self.tweets[id]['tweet']
        else:
            return None
    
    def getTweetCount(self,id):
        return self.tweets[id]['count']
    
    def countTweets(self):
        return len(self.tweets)
    
    # return a sorted list of tupes of the form (id,count), with the occurrence counts sorted in decreasing order
    def mostFrequent(self):
        ps = []
        for t,entry in self.tweets.items():
            count = entry['count']
            ps.append((t,count))  
        ps.sort(key=lambda x: x[1],reverse=True)
        return ps
    
    # reeturns tweet IDs as a set
    def getIds(self):
        return set(self.tweets.keys())
    
    # save the tweets to a file
    def saveTweets(self,filename):
        json_data =jsonpickle.encode(self.tweets)
        with open(filename,'w') as f:
            json.dump(json_data,f)
    
    # read the tweets from a file 
    def readTweets(self,filename):
        with open(filename,'r') as f:
            json_data = json.load(f)
            incontents = jsonpickle.decode(json_data)   
            self.tweets=incontents
        
    def getSearchTerm(self,id):
        return self.tweets[id]['searchTerm']
    
    def getSearchTime(self,id):
        return self.tweets[id]['searchTime']
    
    def getText(self,id):
        tweet = self.getTweet(id)
        text=tweet['full_text']
        if 'retweeted_status'in tweet:
            original = tweet['retweeted_status']
            text=original['full_text']
        return text
                
    def addCode(self,id,code):
        tweet=self.getTweet(id)
        if 'codes' not in tweet:
            tweet['codes']=set()
        tweet['codes'].add(code)
        
   
    def addCodes(self,id,codes):
        for code in codes:
            self.addCode(id,code)
        
 
    def getCodes(self,id):
        tweet=self.getTweet(id)
        return tweet['codes']
    
    # NEW -ROUTINE TO GET PROFILE
    def getCodeProfile(self):
        summary={}
        for id in self.tweets.keys():
            tweet=self.getTweet(id)
            if 'codes' in tweet:
                for code in tweet['codes']:
                    if code not in summary:
                            summary[code] =0
                    summary[code]=summary[code]+1
        sortedsummary = sorted(summary.items(),key=operator.itemgetter(0),reverse=True)
        return sortedsummary

In [4]:
smoking=Tweets()
smoking.readTweets("tweets-smoking.json")
smoking.countTweets()

100

In [133]:
vaping=Tweets()
vaping.readTweets("tweets-vaping.json")
vaping.countTweets()

100

# ok. trying categorization from spaCy examples

https://github.com/explosion/spaCy/blob/master/examples/training/train_textcat.py

In [6]:
nlp =spacy.blank("en")

In [7]:
textcat = nlp.create_pipe('textcat')

In [8]:
 nlp.add_pipe(textcat, last=True)

See - https://spacy.io/api/textcategorizer for info on text cat

In [9]:
textcat.add_label("POSITIVE")

1

In [10]:
import thinc.extra.datasets

In [11]:
from spacy.util import minibatch, compounding
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path

## load data

In [12]:
train_data, _ = thinc.extra.datasets.imdb()

In [13]:
random.shuffle(train_data)

In [14]:
len(train_data)

25000

In [15]:
train_data[1]

('"Welcome to Collinwood" is kind of a disaster. Considering the people involved, it should\'ve been multiple times better.\n\n\n\nWatching it, if you\'re at least somewhat attached to the faith that it\'ll get better, will probably make you cry. It\'s one of those movies that had potential, but was robbed of this potential thanks to a terrible script and some bad acting, not to mention the strangely annoying and unnecessary George Clooney character and the guy who reminds me of Richard Dreyfuss but about whom I care so little that I don\'t even want to know his name.\n\n\n\nThe film\'s only saving grace is the weird con vocabulary it introduces. I found myself thinking of it time and time again as I watched more crime capers. This is the only reason I gave the film a 3. The plot is boring, the characters are neurotic, needlessly offensive, and highly unlikable. They are in a constant state of agonizing stress and they\'re all so irritating that I celebrated their obstacles. They yell 

In [16]:
 train_data = train_data[-2000:]

In [17]:
train_data[0]

('After a fairly lengthy partially pixelated nude shower scene, we\'re off to the races for this "Blair Witch Project"-esquire horror film about three girlfriends venturing to a desolate cabin deep in the woods to get away from their hectic lives for a girls\' weekend out and smoke pot. They meet two guys who seem friendly enough, so they drink and tell ghost stories, until late in the movie some of them get picked off.\n\n\n\nThis is a fairly slow movie, with needlessly drawn out \'suspense\' scenes, the bad acting can\'t carry the myriad of scenes where nothing happens but mindless banter, and the movie as a whole is a dud, a deathly-boring dud at that. Nothing at all happens until the last half hour and when it did I was to numb to really care.\n\n\n\nEye Candy: Ashley Totin shows T&A; Evy Lutzky gets topless briefly; and Jennifer Hart shows her right tit \n\n\n\nMy Grade: D',
 0)

split tuples into exts and labels

In [18]:
texts, labels = zip(*train_data)

In [19]:
texts[0]

'After a fairly lengthy partially pixelated nude shower scene, we\'re off to the races for this "Blair Witch Project"-esquire horror film about three girlfriends venturing to a desolate cabin deep in the woods to get away from their hectic lives for a girls\' weekend out and smoke pot. They meet two guys who seem friendly enough, so they drink and tell ghost stories, until late in the movie some of them get picked off.\n\n\n\nThis is a fairly slow movie, with needlessly drawn out \'suspense\' scenes, the bad acting can\'t carry the myriad of scenes where nothing happens but mindless banter, and the movie as a whole is a dud, a deathly-boring dud at that. Nothing at all happens until the last half hour and when it did I was to numb to really care.\n\n\n\nEye Candy: Ashley Totin shows T&A; Evy Lutzky gets topless briefly; and Jennifer Hart shows her right tit \n\n\n\nMy Grade: D'

In [20]:
labels[0]

0

In [21]:
cats = [{'POSITIVE': bool(y)} for y in labels]

## train on 80% of data

In [22]:
split = int(len(train_data) * 0.8)

In [23]:
split

1600

In [24]:
 (train_texts, train_cats), (dev_texts, dev_cats) = (texts[:split], cats[:split]), (texts[split:], cats[split:])

# ok. now have train texts and cats, along with dev..

rejoin them

In [25]:
train_data = list(zip(train_texts,
                          [{'cats': cats} for cats in train_cats]))

In [26]:
train_data[0]

('After a fairly lengthy partially pixelated nude shower scene, we\'re off to the races for this "Blair Witch Project"-esquire horror film about three girlfriends venturing to a desolate cabin deep in the woods to get away from their hectic lives for a girls\' weekend out and smoke pot. They meet two guys who seem friendly enough, so they drink and tell ghost stories, until late in the movie some of them get picked off.\n\n\n\nThis is a fairly slow movie, with needlessly drawn out \'suspense\' scenes, the bad acting can\'t carry the myriad of scenes where nothing happens but mindless banter, and the movie as a whole is a dud, a deathly-boring dud at that. Nothing at all happens until the last half hour and when it did I was to numb to really care.\n\n\n\nEye Candy: Ashley Totin shows T&A; Evy Lutzky gets topless briefly; and Jennifer Hart shows her right tit \n\n\n\nMy Grade: D',
 {'cats': {'POSITIVE': False}})

In [27]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']

In [28]:
nlp.disable_pipes(*other_pipes)

[]

In [29]:
optimizer = nlp.begin_training()

In [30]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 1e-8  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 1e-8  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score}

In [31]:
print('{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F'))
for i in range(2):
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_data, size=compounding(4., 32., 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(texts, annotations, sgd=optimizer, drop=0.2,losses=losses)
    with textcat.model.use_params(optimizer.averages):
        # evaluate on the dev data split off in load_data()
        scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}'.format(losses['textcat'], scores['textcat_p'],scores['textcat_r'], scores['textcat_f']))
            

LOSS 	  P  	  R  	  F  
70.829	0.842	0.783	0.812
24.861	0.846	0.803	0.824


In [32]:
### questions - what does minibatch do? 
### what does nlp.update do? 
### what does the user_params do? 


### minibatch 

In [33]:
 batches = minibatch(train_data, size=compounding(4., 32., 1.001))

generates subsets.
NLP update updates the models. 
what about evaluate?

In [34]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 1e-8  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 1e-8  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score}

In [35]:
tokenizer= nlp.tokenizer

In [36]:
docs=(tokenizer(text) for text in dev_texts)

In [37]:
docs

<generator object <genexpr> at 0x10c303a40>

In [38]:
enumerated=enumerate(textcat.pipe(docs))

In [39]:
elist=list(enumerated)

In [40]:
i,doc=elist[0]

In [41]:
gold=dev_cats[i]

In [42]:
gold

{'POSITIVE': False}

In [43]:
doc.cats

{'POSITIVE': 0.008255098946392536}

####  thus, gets a +1.
#### ahah. goes through all labels assocaited with the item in the training  data, and see how the tokenizer calls them. 


ok. so this is a bit opaque.  is sci-kit any clearer?

Trying the scikit spacy combination here ---

https://nicschrading.com/project/Intro-to-NLP-with-spaCy/

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
import string
import re

In [46]:
train = ["I love space. Space is great.", "Planets are cool. I am glad they exist in space", 
        "lol @twitterdude that is gr8", "twitter &amp; reddit are fun.", 
        "Mars is a planet. It is red.", "@Microsoft: y u skip windows 9?", 
        "Rockets launch from Earth and go to other planets.", "twitter social media &gt; &lt;", 
        "@someguy @somegirl @twitter #hashtag", "Orbiting the sun is a little blue-green planet."]
labelsTrain = ["space", "space", "twitter", "twitter", "space", "twitter", "space", "twitter", "twitter", "space"]

test = ["i h8 riting comprehensibly #skoolsux", "planets and stars and rockets and stuff"]
labelsTest = ["twitter", "space"]

In [47]:
vectorizer = CountVectorizer()

In [49]:
vectorizer.transform(train)

NotFittedError: CountVectorizer - Vocabulary wasn't fitted.

In [50]:
vectorizer.fit(train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [51]:
print(vectorizer.vocabulary_)

{'love': 23, 'space': 39, 'is': 18, 'great': 13, 'planets': 31, 'are': 3, 'cool': 5, 'am': 0, 'glad': 10, 'they': 43, 'exist': 7, 'in': 17, 'lol': 22, 'twitterdude': 46, 'that': 41, 'gr8': 12, 'twitter': 45, 'amp': 1, 'reddit': 33, 'fun': 9, 'mars': 25, 'planet': 30, 'it': 19, 'red': 32, 'microsoft': 27, 'skip': 35, 'windows': 47, 'rockets': 34, 'launch': 20, 'from': 8, 'earth': 6, 'and': 2, 'go': 11, 'to': 44, 'other': 29, 'social': 36, 'media': 26, 'gt': 15, 'lt': 24, 'someguy': 38, 'somegirl': 37, 'hashtag': 16, 'orbiting': 28, 'the': 42, 'sun': 40, 'little': 21, 'blue': 4, 'green': 14}


In [52]:
vectorizer.get_feature_names()

['am',
 'amp',
 'and',
 'are',
 'blue',
 'cool',
 'earth',
 'exist',
 'from',
 'fun',
 'glad',
 'go',
 'gr8',
 'great',
 'green',
 'gt',
 'hashtag',
 'in',
 'is',
 'it',
 'launch',
 'little',
 'lol',
 'love',
 'lt',
 'mars',
 'media',
 'microsoft',
 'orbiting',
 'other',
 'planet',
 'planets',
 'red',
 'reddit',
 'rockets',
 'skip',
 'social',
 'somegirl',
 'someguy',
 'space',
 'sun',
 'that',
 'the',
 'they',
 'to',
 'twitter',
 'twitterdude',
 'windows']

In [54]:
vectorizer.vocabulary_.get("windows")

47

In [55]:
vector=vectorizer.transform(train)

In [56]:
print(vector.shape)
print(type(vector))
print(vector.toarray())

(10, 48)
<class 'scipy.sparse.csr.csr_matrix'>
[[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 2 0 0 0 0 0 0 0 0]
 [1 0 0 1 0 1 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 1 0 0 0 0 1 0]
 [0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1
  0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0
  0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
  1 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 1 1 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 

In [57]:
len(train)

10

https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(train)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)

{'love': 23, 'space': 39, 'is': 18, 'great': 13, 'planets': 31, 'are': 3, 'cool': 5, 'am': 0, 'glad': 10, 'they': 43, 'exist': 7, 'in': 17, 'lol': 22, 'twitterdude': 46, 'that': 41, 'gr8': 12, 'twitter': 45, 'amp': 1, 'reddit': 33, 'fun': 9, 'mars': 25, 'planet': 30, 'it': 19, 'red': 32, 'microsoft': 27, 'skip': 35, 'windows': 47, 'rockets': 34, 'launch': 20, 'from': 8, 'earth': 6, 'and': 2, 'go': 11, 'to': 44, 'other': 29, 'social': 36, 'media': 26, 'gt': 15, 'lt': 24, 'someguy': 38, 'somegirl': 37, 'hashtag': 16, 'orbiting': 28, 'the': 42, 'sun': 40, 'little': 21, 'blue': 4, 'green': 14}
[2.70474809 2.70474809 2.70474809 2.29928298 2.70474809 2.70474809
 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809
 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809
 1.78845736 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809
 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809 2.70474809
 2.29928298 2.29928298 2.70474809 2.70474809 2.70474809 2.70474809
 

In [60]:
vector = vectorizer.transform(train)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(10, 48)
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.43323568 0.         0.         0.         0.
  0.28646791 0.         0.         0.         0.         0.43323568
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.73657982 0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.34989927 0.         0.         0.29744635 0.         0.34989927
  0.         0.34989927 0.         0.         0.34989927 0.
  0.         0.         0.         0.         0.         0.34989927
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.29744635 0.         0.         0.         0.
  0.         0.         0.         0.29744635 0.         0.
  0.         0.34989927 0.         0.         0.         0

ok. now -- how can I train on this? 
and how can I pass in my spacy tokens?

In [74]:
nlp =spacy.load("en")
def tokenizeText(sample):

    # get the tokens using spaCy
    tokens = nlp(sample)

    # stoplist the tokens
    tokens = [tok for tok in tokens if tok.is_stop ==False]
    # lemmatize
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas


    # remove large strings of whitespace
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")

    return tokens

In [75]:
tokenizeText(train[0])

['i', 'love', 'space', '.', 'space', 'great', '.']

In [70]:
tokens=nlp(train[0])

In [71]:
tokens

I love space. Space is great.

In [83]:
tokens[0].is_stop

False

# plan

1. Go back to https://nicschrading.com/project/Intro-to-NLP-with-spaCy/

1.1 create NLP as per part 3 with hashtag tokenizer.

1.2. wrap that in a tokenizer that calls the spacy NLP parser and tokenizes with stops, etc.

1.3 use that tokenizer in a tf-df

1.4 follow that up with classifier. 

1.5 test


In [85]:
nlp = spacy.load('en')
def hashtag_pipe(doc):
    merged_hashtag = True
    while merged_hashtag == True:
        merged_hashtag = False
        for token_index,token in enumerate(doc):
            if token.text == '#':
                try:
                    nbor = token.nbor()
                    start_index = token.idx
                    end_index = start_index + len(token.nbor().text) + 1
                    if doc.merge(start_index, end_index) is not None:
                        merged_hashtag = True
                        break
                except:
                    pass
    return doc
nlp.add_pipe(hashtag_pipe,first=True)

In [86]:
def tokenizeText(sample):
    tokens = nlp(sample)
    
     # lemmatize
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    
    # stoplist the tokens
    tokens = [tok for tok in tokens if tok.is_stop == False]
    
    # remove large strings of whitespace
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")

    return tokens

1.3 - set up tokenizer in a vectorizer.

In [88]:
vectorizer = TfidfVectorizer(tokenizer=tokenizeText)

1.4 now I would do vectorizer. fit - need good input and labels..

In [89]:
smoking

In [90]:
vaping

## ok. to do this I think the following needs to happen:

1. take all tweets from smoking and vaping and put them into a list
2. make a similar list of smoking and vaping categories...

In [94]:
ids=smoking.getIds()

In [97]:
random.shuffle(ids)

TypeError: 'set' object does not support indexing

In [98]:
ids = list(ids)

In [99]:
ids[0]

'974316981229846529'

In [101]:
random.shuffle(ids)

In [103]:
ids[0]

'974317099186204672'

In [107]:
splitRatio=0.8
split = int(len(ids)*splitRatio)

In [108]:
split

80

In [109]:
trainSmoke=ids[:split]
testSmoke=ids[split:]

In [110]:
len(trainSmoke)

80

In [111]:
len(testSmoke)

20

In [114]:
smoking.getSearchTerm(trainSmoke[0])

'smoking'

In [118]:
def getTweetsAndCats(tweets,ids):
    texts=[]
    cats=[]
    for i in ids:
        text = tweets.getText(i)
        cat = tweets.getSearchTerm(i)
        texts.append(text)
        cats.append(cat)
    return texts,cats

def getTestTrain(tweets,splitFactor=0.8):
    ids =list(tweets.getIds())
    random.shuffle(ids)
    split = int(len(ids)*splitFactor)
    trainIds=ids[:split]
    testIds=ids[split:]
    trainTweets,trainCats = getTweetsAndCats(tweets,trainIds)
    testTweets,testCats = getTweetsAndCats(tweets,testIds)
    return trainTweets,trainCats,testTweets,testCats

In [125]:
trainST,trainSC,testST,testSC = getTestTrain(smoking)

In [134]:
trainVT,trainVC,testVT,testVC = getTestTrain(vaping)

KeyError: 'full_text'

In [128]:
traint = trainST+trainVT
trainc = trainSC+trainVC
testt = testST+testVT
testc=testSC+testVC

In [130]:
len(trainc)

160